In [81]:
from dotenv import load_dotenv #install dotenv - pip install python-dotenv(to load environment variables)
load_dotenv() #loading env variables
from pinecone import Pinecone,ServerlessSpec #pip install "pinecone-client[grpc]"
import os
from openai import OpenAI 
#pip install -r requirments.txt

In [82]:

pc = Pinecone(api_key = os.getenv("PINECONE_API_KEY"))
print(pc)
index_name = "rag"
if index_name not in pc.list_indexes().names():
    pc.create_index(
    name=index_name,
    dimension = 1536,
    metric = "cosine",
    spec = ServerlessSpec(cloud = "aws",region = "us-east-1")
    
)

In [85]:
import json
data = json.load(open("reviews.json"))
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
print(client)
processed_data = []

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review']+str(review["stars"])+review["subject"],
        model = "text-embedding-3-small"
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values": embedding,
        "id" : review['professorname'],
        "metadata":{
            "review":review['review'],
            "subject":review['subject'],
            "stars":review['stars']
            
        }
        
    })

index = pc.Index("rag")
upsert_response = index.upsert(
    vectors=processed_data,
    namespace="ns1",
)
print(f"Upserted count: {upsert_response['upserted_count']}")

# Print index statistics

text = "suggest A professor who teach mathematics"
response = client.embeddings.create(
    input=text,
    model = "text-embedding-3-small"
)
myinput = response.data[0].embedding
query_results1 = index.query(
    namespace="ns1",
    vector=myinput,
    top_k=3,
    include_values=True,
    include_metadata=True
)
for result in query_results1['matches']:
    print(result["id"])
    print(result["metadata"])
    


Upserted count: 10
Prof. Jane Smith
{'review': 'Prof. Smith is very knowledgeable, but her lectures can be a bit fast-paced. Overall, a good experience.', 'stars': 4.0, 'subject': 'Mathematics'}
Prof. Kevin Moore
{'review': 'Prof. Moore is an outstanding professor. His dynamic teaching style and real-world examples make sociology incredibly interesting.', 'stars': 5.0, 'subject': 'Sociology'}
Dr. John Doe
{'review': "Dr. Doe's lectures are always insightful and engaging. He explains complex concepts with ease.", 'stars': 5.0, 'subject': 'Computer Science'}
